# Boilerplate

There's probably a better way than what follows

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

What can we run?

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['EndpointW',
 'PacketW',
 'RequestActionW',
 'TransportInterface',
 'new_transport_interface']

## Transport Setup

Create a new transport interface, asynchronously. This uses the Python `asyncio` event loop, which a Jupyter Notebook already has running.

In [3]:
tiface_fut = new_transport_interface("0.0.0.0", 12346)
tiface_fut

<Future pending cb=[<builtins.PyDoneCallback object at 0x7fcde1eb7530>()]>

Verify the `Future` has completed.

In [4]:
assert tiface_fut.done()

Call the `result` method to get the result when it's complete.

In [5]:
tiface = tiface_fut.result()
tiface

TransportInterface{ transport: Some(<Transport>),   cmd_tx: Sender { chan: Tx { inner: Chan { tx: Tx { block_tail: 0x7fcda40133c0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } },   response_rx: Receiver { chan: Rx { inner: Chan { tx: Tx { block_tail: 0x7fcda4013fb0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } },   notify_rx: Receiver { chan: Rx { inner: Chan { tx: Tx { block_tail: 0x7fcda40143a0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } } }

### Run the Transport in background
Here is the fun part :)

In [6]:
run_fut = tiface.run()
run_fut

<Future pending cb=[<builtins.PyDoneCallback object at 0x7fcde1eb7370>()]>

If the Future completed right away, it means something's fishy.

In [9]:
assert not run_fut.done()

Note the `transport: None` at the beginning of the following `repr` output. It is `None` because `run()` takes the `Transport`.

In [10]:
tiface

TransportInterface{ transport: None,   cmd_tx: Sender { chan: Tx { inner: Chan { tx: Tx { block_tail: 0x7fcda40133c0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } },   response_rx: Receiver { chan: Rx { inner: Chan { tx: Tx { block_tail: 0x7fcda4013fb0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } },   notify_rx: Receiver { chan: Rx { inner: Chan { tx: Tx { block_tail: 0x7fcda40143a0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } } }

## Transport Operation

In [11]:
# TODO